In [1]:
import time
import datetime
import os
import json

In [4]:
print("hello allmind")

hello allmind


-----

模块类

In [ ]:
class Plugin:
    def __init__(self, name):
        self.name = name

    def run(self):
        print(f"Plugin {self.name} is running.")

In [ ]:
class Message:
    def __init__(self, content):
        self.timestamp = datetime.datetime.now()
        self.content = content

In [ ]:
class TerminalIO:
    def __init__(self, prompt="> "):
        self.prompt =  prompt

    def input(self, prompt):
        user_input = input(prompt)
        return user_input

    def output(self, message):
        print(message)

In [ ]:
class LLM:
    def __init__(self, api_key, model, system_prompt):
        self.api_key = api_key
        self.model = model
        self.system_prompt = system_prompt

    def plain_chat(message_json):
        messages = LLMMessage(content)
        return message.content
    
class LLMMessage:
    def __init__(self, role, content):
        self.role = role
        self.content = content

    def to_dict(self):
        return {
            "role": self.role,
            "content": self.content
        }
        
    def to_json(self):
        return json.dumps(self.to_dict())
    
    def from_json(json_str):
        data = json.loads(json_str)
        return LLMMessage(data['role'], data['content'])

------------

先不搞这么清楚，就先按流程跑完，跑完在说模块化的事情

In [ ]:
from openai import OpenAI

# 从文件中读取API密钥
def read_api_key_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            keys = json.load(file)  # 读取文件内容
            return keys["deepseek"]
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except IOError:
        print(f"Error: Could not read the file {file_path}.")  
        
# 可调参数
api_key = read_api_key_from_file("keys.json")
if api_key is None:
    print("API key not found in the file.")
else:   
    print("API key loaded successfully.")
base_url = "https://api.deepseek.com"
model = "deepseek-chat"
system_prompt = "You are a helpful assistant."
    
# 使用API密钥创建OpenAI客户端
client = OpenAI(api_key=api_key, base_url=base_url)

# 输入输出循环
while True:
    # 获取用户输入
    user_input = input("Enter your message (or 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    
    # 调用API进行聊天
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        stream=False
    )
    
    # 输出响应内容
    print(response.choices[0].message.content)

API key loaded successfully.
你好！😊 很高兴见到你！有什么我可以帮你的吗？
我是DeepSeek Chat，由深度求索公司创造的智能AI助手！✨ 我的使命是帮助你解答问题、提供信息、陪你聊天，或者一起探索各种有趣的话题。无论是学习、工作，还是日常生活中的小疑问，都可以找我聊聊！😊  

有什么我可以帮你的吗？


------
拆分io 和 LLM

In [ ]:
# LLM 相关代码
# jupyter notebook 需要使用 nest_asyncio 来解决启动问题
import nest_asyncio
import uvicorn
from threading import Thread
# llm plugin
from fastapi import FastAPI, Request
from pydantic import BaseModel
from openai import OpenAI
import json


# 读取API Key
with open("keys.json", "r") as f:
    api_key = json.load(f)["deepseek"]

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
app = FastAPI()

class ChatRequest(BaseModel):
    messages: list  # [{"role": "user", "content": "..."}]

@app.post("/chat")
def chat(req: ChatRequest):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "system", "content": "You are a helpful assistant."}] + req.messages,
        stream=False
    )
    return {"reply": response.choices[0].message.content}

# 允许在当前的事件循环中运行 asyncio
nest_asyncio.apply()
# 启动 FastAPI 应用
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
thread = Thread(target=server.run)
thread.start()

INFO:     Started server process [60364]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52273 - "GET /chat HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:52273 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52265 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:52469 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:52493 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:52523 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:52566 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:52602 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:62672 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:62770 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:62848 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:58042 - "POST /read_file HTTP/1.1" 404 Not Found


In [ ]:
# io相关代码，在独立终端中运行
# io_client/cli.py
import requests

def main():
    messages = []
    print("LLM CLI Demo (type 'exit' to quit)")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        messages.append({"role": "user", "content": user_input})
        
        res = requests.post("http://localhost:8000/chat", json={"messages": messages})
        reply = res.json()["reply"]
        print("Bot:", reply)
        
        messages.append({"role": "assistant", "content": reply})

if __name__ == "__main__":
    main()

-------------
增加一个planner做中间商，协调调用不同tool

In [ ]:
# Planner

planner_prompt_template = """
你是一个专业的计划制定者，负责根据用户的自然语言需求，制定信息收集与工具调用的策略，逐步完成复杂任务的拆解与执行。你具备强大的推理和工具规划能力。

你遵循以下工作流程：

1. **理解任务需求**：从用户输入中提取任务目标和上下文信息。
2. **规划信息收集**：判断完成任务所需的信息和资源。
3. **选择工具**：查阅可用工具列表，选取最适合的信息获取或操作手段。
4. **获取工具说明**：使用内置函数 `get_tool_args(tool_name)` 获取工具参数结构与说明。
5. **构造工具参数**：根据当前任务上下文和目标，补全工具所需参数, 整合为tool_args_json_str。
6. **调用工具**：使用 `call_tool(tool_name, tool_args_json_str)` 调用该工具，并获取结果。
7. **基于结果迭代优化**：根据工具返回结果调整下一步计划，可能需多轮工具调用。
8. **无法完成时交互澄清**：若当前信息不足，使用 `plan_mode_respond` 与用户对话，澄清需求或补充上下文。
9. **输出结果**：任务达成后，调用 `plan_mode_respond` 输出最终回复。

---

你可以调用的工具如下：

{tool_list}

工具使用示例：

```python
# 获取工具参数结构与说明
get_tool_args("search_wikipedia")

# 使用返回的结构构造参数并调用工具
call_tool("search_wikipedia", '{"query": "量子力学的基本原理","language": "zh"}')
若你无法确定接下来的工具参数或目标，请使用 plan_mode_respond 继续与用户对话。
"""

class Planner:
    def __init__(self):
        base_url = "https://api.deepseek.com"
        api_key = read_api_key_from_file("keys.json")
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.tools = {}
        self.system_prompt = planner_prompt_template
        self.model = "deepseek-chat"
        
    def add_tool(self, tool):
        self.tools[tool["name"]] = tool
        tool_list = "\n".join([f"- {tool['name']}: {tool['description']}" for tool in self.tools.values()])
        self.system_prompt = self.system_prompt.format(tool_list=tool_list)
        
    def get_tool_args(self, tool_name):
        # """
        # {
        #     "name" : "get_tool_args",
        #     "description" : "获取工具参数结构与说明",
        #     "arguments" : {
        #         "tool_name" : {
        #             "description" : "工具名称",
        #             "required" : true,
        #             "default" : null
        #         }
        #     },
        #     "url" : "http://localhost:18000/planner/get_tool_args",
        #     "method" : "POST"
        # }
        # """
        # 获取工具参数结构与说明
        if tool_name in self.tools:
            return json.dumps(self.tools[tool_name].description)
        else:
            return None
        
    def call_tool(self, tool_name, tool_args_json_str):
        # """
        # {
        #     "name" : "call_tool",
        #     "description" : "根据工具名称和参数调用工具",
        #     "arguments" : {
        #         "tool_name" : {
        #             "description" : "工具名称",
        #             "required" : true,
        #             "default" : null
        #         },
        #         "tool_args_json_str" : {
        #             "description" : "工具参数的JSON字符串",
        #             "required" : true,
        #             "default" : null
        #         }
        #     },
        #     "url" : "http://localhost:18000/planner/call_tool",
        #     "method" : "POST"
        # }
        # """
        # 调用工具
        if tool_name in self.tools:
            tool = self.tools[tool_name]
            return tool.call(tool_args_json_str)
        else:
            return None

    

In [19]:
# 在python中，我正在设计LLM functioncall功能，这项功能中有两个参与者：LLM和tools，LLM根据tools的名称、描述、和参数选择合适的tools完成任务
class Tool:
    def __init__(self, config_str):
        config = json.loads(config_str)
        self.url = config["url"]
        self.method = config["method"]
        self.name = config["name"]
        self.description = config["description"]
        self.arguments = config["arguments"]
        
    def call(self, arguments_str):
        # 解析参数，并检验参数是否符合要求
        arguments = json.loads(arguments_str)
        for arg_name, arg_value in arguments.items():
            if arg_name not in self.arguments:
                raise ValueError(f"Argument {arg_name} is not valid for this tool.")
            arg_config = self.arguments[arg_name]
            if arg_config["required"] and arg_value is None:
                raise ValueError(f"Argument {arg_name} is required but not provided.")
        # 使用requests库发送HTTP请求
        response = requests.request(self.method, self.url, json=arguments)
        if response.status_code != 200:
            raise ValueError(f"Tool call failed with status code {response.status_code}: {response.text}")
        return response.json()

# test_tool_config_str = """{
#     "name" : "read_file",
#     "description" : "Read a file from the local filesystem",
#     "arguments" : {
#         "file_path" : {
#             "description" : "The path to the file to read",
#             "required" : true,
#             "default" : null
#         },
#         "encoding" : {
#             "description" : "The encoding of the file",
#             "required" : false,
#             "default" : "utf-8"
#         }
#     },
#     "url" : "http://localhost:8000/read_file",
#     "method" : "POST"
# }
# """

# test_tool_config_str = """{
#     "name" : "read_file",
#     "description" : "Read a file from the local filesystem",
#     "arguments" : {
#         "file_path" : {
#             "description" : "The path to the file to read",
#             "required" : true,
#             "default" : null
#         },
#         "encoding" : {
#             "description" : "The encoding of the file",
#             "required" : false,
#             "default" : "utf-8"
#         }
#     },
#     "url" : "http://localhost:8000/read_file",
#     "method" : "POST"
# }
# """

test_tool_config_str = """{
    "name" : "read_file",
    "description" : "Read a file from the local filesystem",
    "arguments" : {
        "file_path" : {
            "description" : "The path to the file to read",
            "required" : true,
            "default" : null
        },
        "encoding" : {
            "description" : "The encoding of the file",
            "required" : false,
            "default" : "utf-8"
        }
    },
    "url" : "http://localhost:8000/read_file",
    "method" : "POST"
}
"""
# import json
t = Tool(test_tool_config_str)
json.dumps(t.arguments)
# t.call('{"file_path": "test.txt", "encoding": "utf-8"}')



'{"file_path": {"description": "The path to the file to read", "required": true, "default": null}, "encoding": {"description": "The encoding of the file", "required": false, "default": "utf-8"}}'